<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/Images/logocce.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1> Pipeline de Datos de Contratación Pública
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.dnp.gov.co/img/logoNuevo.jpg" style="width: 200px;"></a></div>
</div>


## 1. IDENTIFICACIÓN DEL INSUMO

|||
|:--|:--|
|**Fecha**|Abril 2023|
|**Ciudad**|Bogotá D.C.|
|**Esquema de presentación del insumo**|Cuaderno Jupyter|
|**Título del insumo**| **Pipeline de Datos de Contratación Pública**|
|**Descripción y alcance**|Script para la consulta de bases y exploración de bases en el DATAVAULT.|
|**Periodicidad del insumo**|único|
|**Solicitante**|No aplica|
|**Versión del insumo**|Final|

## 2. DESTINO Y AUTORES DEL INFORME / INSUMO

|||
|:--|:--|
|**Destinatario**|<table align='left'><tr><td>*Nombre:*</td> <td>Equipo analítica EMAE</td></tr> <tr><td>*Cargo:*</td> <td>NA</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE</td></tr></table>|
|**Autores**|<table><tr><td>*Nombre:*</td> <td>Equipo de Datos -GAEC</td></tr><tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|
|**Aprobación**|<table><tr><td>*Nombre:*</td> <td>Ricardo Suarez</td></tr> <tr><td>*Cargo:*</td> <td>Subdirector Estudios de Mercado y Abastecimiento Estratégico</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|

## Carga de librerías y ambiente de trabajo

En esta sección se cargan las librerías necesarias para el desarrollo del script y se establecen los parámetros de trabajo.


In [1]:
import pyodbc 
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
### Paquetes usados para la exploración de datos

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import warnings
warnings.filterwarnings('ignore')


<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
ruta_DATA="/dropbox/Dropbox/CCE-projects/Large Data/SECOP DA/"

In [ ]:
DF_Consulta=pd.read_parquet(ruta_DATA+"SECOP_I_II.parquet")

In [9]:
## Estado de los contratos

# Diccionario de estandarización de estados contractuales
estado_contractual = {
    'En ejecución': 'En ejecución',
    'Modificado': 'Modificado',
    'Celebrado': 'Celebrado',
    'terminado': 'Terminado',
    'Liquidado': 'Liquidado',
    'Cerrado': 'Cerrado',
    'Activo': 'Activo',
    'Terminado sin Liquidar': 'Terminado sin Liquidar',
    'cedido': 'Cedido',
    'Suspendido': 'Suspendido',
    'Convocado': 'Convocado',
    'Adjudicado': 'Adjudicado'
}

# Corregir los estados contractuales utilizando el diccionario
DF_Consulta['Estado'] = DF_Consulta['Estado'].replace(estado_contractual)
DF_Consulta['Estado'].value_counts()


Estado
Celebrado                                      1711196
En ejecución                                    993408
Liquidado                                       721506
Modificado                                      497127
Terminado                                       295685
Cerrado                                         137549
Activo                                          102023
Terminado sin Liquidar                           75416
Cedido                                           15355
Suspendido                                        2763
Convocado                                          579
Adjudicado                                         107
Terminado Anormalmente después de Convocado          5
enviado Proveedor                                    2
Name: count, dtype: int64

In [10]:
### Ajustemos las entidades

# Carga de MDM de entidades

MDM_Entidades = pd.read_excel('../../../Sample_Data/Auxiliar/MDM_Total.xlsx')

## 

In [11]:
MDM_SII=MDM_Entidades.drop_duplicates(subset=['Codigo_Entidad_SECOP_II'], keep='first')
MDM_SI=MDM_Entidades.drop_duplicates(subset=['Codigo_Entidad_SECOP_I'], keep='first')

In [ ]:
### Agregamos el código de entidad y corregimos el nombre de la entidad usando el MDM segun la fuente

# Agregamos el código de entidad
cod_ent=[]
rev_MDM=[]
DF_Consulta['NIT_MDM']=np.nan
for ind in DF_Consulta.index:
    if DF_Consulta['Fuente'][ind]=='SECOP I':
        cod_entidad=MDM_SI[MDM_SI['Codigo_Entidad_SECOP_I']==DF_Consulta['ID entidad Plataforma'][ind]]
        if cod_entidad.empty or  len(cod_entidad)>1:
            rev_MDM.append([DF_Consulta['Entidad'][ind],DF_Consulta['ID entidad Plataforma'][ind],'SECOP I'])    
            cod_ent.append(np.nan)
        else:
            cod_ent.append(cod_entidad['Codigo_Entidad'].values[0])
            DF_Consulta['NIT_MDM'][ind]=cod_entidad['NIT_SECOP'].values[0]
        
    elif DF_Consulta['Fuente'][ind]=='SECOP II':
        cod_entidad=MDM_SII[MDM_SII['Codigo_Entidad_SECOP_II']==DF_Consulta['ID entidad Plataforma'][ind]]
        if cod_entidad.empty or  len(cod_entidad)>1:
            rev_MDM.append([DF_Consulta['Entidad'][ind],DF_Consulta['ID entidad Plataforma'][ind],'SECOP II'])
            cod_ent.append(np.nan)
        else:
            cod_ent.append(cod_entidad['Codigo_Entidad'].values[0])
            DF_Consulta['NIT_MDM'][ind]=cod_entidad['NIT_SECOP'].values[0]
DF_Consulta['Codigo Entidad']=cod_ent

Error_MDM=pd.DataFrame(rev_MDM,columns=['Entidad','ID entidad Plataforma','Fuente'])
Error_MDM.drop_duplicates(keep='first',inplace=True)


In [16]:
Error_MDM.to_excel('../../../Sample_Data/Processed/Errores MDM.xlsx',index=False)

In [17]:
DF_Consulta['Entidad']=DF_Consulta['Entidad'].str.upper()

In [18]:
## Exploración de los datos
# Vemos el tamaño del dataset
print("El dataset tiene", DF_Consulta.shape[0], "filas y", DF_Consulta.shape[1], "columnas")

# Vemos los tipos de datos
print(DF_Consulta.dtypes)

#Extraemos para este reporte las URL de los contratos

#DF_Consulta.drop('URL', axis=1, inplace=True)

El dataset tiene 4552721 filas y 29 columnas
Unificado
Fuente                                 object
ID Contrato                            object
ID Proceso                             object
ID entidad Plataforma                  object
Entidad                                object
NIT Entidad                            object
Orden                                  object
Modalidad                              object
Estado                                 object
Descripción proceso                    object
Objeto Contrato                        object
Tipo de contrato                       object
Fecha de Firma                 datetime64[ns]
UNSPSC                                 object
Nombre Proveedor                       object
Documento Proveedor                    object
Tipo de documento proveedor            object
Valor del contrato                     object
Departamento Entidad                   object
Ciudad Entidad                         object
Inicio de contrato       

In [19]:
### Arreglamos el orden

DF_Consulta['Orden']=DF_Consulta['Orden'].str.capitalize()
DF_Consulta['Orden'].value_counts()

Orden
Territorial             3742377
Nacional                 775992
Corporación autónoma      27926
No definido                6426
Name: count, dtype: int64

In [20]:
reemplazos_modalidades = {
    'Asociación Público Privada': 'Modalidad Competitiva',
    'Régimen Especial': 'Modalidad Especial',
    'Iniciativa Privada sin recursos públicos': 'Modalidad Especial',
    'Contratación Directa (Ley 1150 de 2007)': 'Modalidad Directa',
    'Contratación Directa Menor Cuantía': 'Modalidad Directa',
    'Contratos y convenios con más de dos partes': 'Modalidad Directa',
    'Otras Formas de Contratación Directa': 'Modalidad Directa',
    'Concurso de diseño Arquitectónico': 'Modalidad Competitiva',
    'Concurso de Méritos Abierto': 'Modalidad Competitiva',
    'Concurso de Méritos con Lista Corta': 'Modalidad Competitiva',
    'Concurso de Méritos con Lista Multiusos': 'Modalidad Competitiva',
    'Licitación Pública': 'Modalidad Competitiva',
    'Licitación obra pública': 'Modalidad Competitiva',
    'Licitación Pública Nacional (BM)': 'Modalidad Competitiva',
    'Selección Abreviada de Menor Cuantía (Ley 1150 de 2007)': 'Modalidad Competitiva',
    'Selección Abreviada del literal h del numeral 2 del artículo 2 de la Ley 1150 de 2007': 'Modalidad Competitiva',
    'Selección Abreviada servicios de Salud': 'Modalidad Competitiva',
    'Subasta': 'Modalidad Competitiva',
    'Contratación Mínima Cuantía': 'Modalidad Competitiva',
    'Invitación ofertas cooperativas o asociaciones de entidades territoriales': 'Modalidad Competitiva',
    'Lista Multiusos': 'Modalidad Competitiva',
    'Llamado a presentar expresiones de interés': 'Modalidad Competitiva',
    'Solicitud de información a los Proveedores': 'Modalidad Competitiva',
    'Contratación régimen especial': 'Modalidad Especial',
    'Contratación régimen especial (con ofertas)': 'Modalidad Especial',
    'Enajenación de bienes con sobre cerrado': 'Modalidad Competitiva',
    'Enajenación de bienes con subasta': 'Modalidad Competitiva',
    'Contratación directa': 'Modalidad Directa',
    'Selección Abreviada de Menor Cuantía': 'Modalidad Competitiva',
    'Mínima cuantía': 'Modalidad Competitiva',
    'Seleccion Abreviada Menor Cuantia Sin Manifestacion Interes': 'Modalidad Competitiva',
    'Licitación pública': 'Modalidad Competitiva',
    'Selección abreviada subasta inversa': 'Modalidad Competitiva',
    'Concurso de méritos abierto': 'Modalidad Competitiva',
    'Contratación Directa (con ofertas)': 'Modalidad Directa',
    'Licitación Pública Acuerdo Marco de Precios': 'Modalidad Competitiva',
    'Licitación pública Obra Publica': 'Modalidad Competitiva',
    'CCE-20-Concurso_Meritos_Sin_Lista_Corta_1Sobre': 'Modalidad Competitiva',
    'Concurso de méritos con precalificación': 'Modalidad Competitiva',
    'CCE-19-Concurso_Meritos_Con_Lista_Corta_1Sobre': 'Modalidad Competitiva'
}

DF_Consulta['Modalidad Corta']=DF_Consulta['Modalidad'].replace(reemplazos_modalidades)
DF_Consulta['Modalidad Corta'].value_counts()

Modalidad Corta
Modalidad Directa        2788339
Modalidad Especial       1363147
Modalidad Competitiva     397881
No Definido                 3354
Name: count, dtype: int64

In [21]:
### Variables de texto largo, las pasamos a mayúsculas
DF_Consulta['Descripción proceso']=DF_Consulta['Descripción proceso'].str.upper()
DF_Consulta['Objeto Contrato']=DF_Consulta['Objeto Contrato'].str.upper()


In [22]:
### Arreglamos Tipo de Contrato
arreglos_tipo_contrato={
    'Prestación de servicios': 'Prestación de Servicios',
    'Suministro': 'Suministros',
    'Suministros': 'Suministros',
    'Otro': 'Otros',
    'Compraventa': 'Compraventa',
    'Decreelaw092/2017': 'Otros',
    'Otro tipo de contrato': 'Otros',
    'Arrendamiento de inmuebles': 'Arrendamiento',
    'Obra': 'Obras',
    'Interventoría': 'Interventoría',
    'Comodato': 'Arrendamiento',
    'Consultoría': 'Servicios',
    'Arrendamiento': 'Arrendamiento',
    'Acuerdo marco de precios': 'Acuerdo Marco de Precios',
    'Concesión': 'Concesión',
    'Arrendamiento de muebles': 'Arrendamiento',
    'Seguros': 'Seguros',
    'Servicios financieros': 'Servicios Financieros',
    'Venta muebles': 'Compraventa',
    'Comisión': 'Comisión',
    'Asociación público privada': 'Asociación Público Privada',
    'Acuerdo marco': 'Acuerdo Marco de Precios',
    'Emprestito': 'Emprestito',
    'Fiducia': 'Fiducia',
    'No definido': 'No definido',
}


DF_Consulta['Tipo de contrato']=DF_Consulta['Tipo de contrato'].str.capitalize().replace(arreglos_tipo_contrato)
DF_Consulta['Tipo de contrato'].value_counts(dropna=False)

Tipo de contrato
Prestación de Servicios       3673216
Suministros                    372853
Otros                          219854
Compraventa                    128643
Obras                           71398
Arrendamiento                   45963
Servicios                       17184
Interventoría                   12086
No especificado                  3994
Seguros                          2996
Acuerdo Marco de Precios         1780
Crédito                           745
Concesión                         569
Servicios Financieros             378
Emprestito                        317
Fiducia                           282
No definido                       129
Asociación Público Privada         98
Comisión                           81
Negocio fiduciario                 74
Agregación de demanda              58
Venta inmuebles                    22
Acuerdo de cooperación              1
Name: count, dtype: int64

In [23]:
DF_Consulta['Fecha de Firma']=pd.to_datetime(DF_Consulta['Fecha de Firma'], format='%d/%m/%Y', errors='coerce')
DF_Consulta['Inicio de contrato']=pd.to_datetime(DF_Consulta['Inicio de contrato'], format='%d/%m/%Y', errors='coerce')
DF_Consulta['Fin de contrato']=pd.to04['Fecha de Firma'].dt.month

In [25]:
import re

In [26]:
DF_Consulta['UNSPSC']=DF_Consulta['UNSPSC'].astype(str)
codigos_limpios = []
alertas_unspc = []
for codigo in DF_Consulta['UNSPSC']:
    codigo_limpio = re.sub('[^0-9]', '', codigo)
    
    if len(codigo_limpio) ==6:
        codigos_limpios.append(codigo_limpio+'00')
        alertas_unspc.append(False)
    elif len(codigo_limpio) ==9:
        codigos_limpios.append(codigo_limpio[1:])
        alertas_unspc.append(False)
    elif len(codigo_limpio) !=8:
        codigos_limpios.append(codigo_limpio)
        alertas_unspc.append(True)
    else:
        codigos_limpios.append(codigo_limpio)
        alertas_unspc.append(False)

DF_Consulta['UNSPSC']=codigos_limpios
DF_Consulta['Alerta UNSPSC']=alertas_unspc

In [27]:
DF_Consulta['Alerta UNSPSC'].value_counts(dropna=False)

Alerta UNSPSC
False    4548529
True        4192
Name: count, dtype: int64

In [28]:
### Proveedores

DF_Consulta['Nombre Proveedor']=DF_Consulta['Nombre Proveedor'].str.upper()
DF_Consulta['Documento Proveedor']=DF_Consulta['Documento Proveedor'].str.upper()
DF_Consulta['Tipo de documento proveedor']=DF_Consulta['Tipo de documento proveedor'].str.upper()

In [29]:
DF_Consulta['Valor del contrato']=DF_Consulta['Valor del contrato'].astype(int)

In [30]:
DF_Consulta['Valor del contrato'].describe()

count    4.552721e+06
mean     1.733811e+08
std      8.905748e+10
min      0.000000e+00
25%      5.000000e+06
50%      1.165800e+07
75%      2.592038e+07
max      1.420769e+14
Name: Valor del contrato, dtype: float64

In [31]:
reemplazos_departamento = {
    'Bogotá DC': 'Bogotá D.C.',
    'Bogotá, D.C.': 'Bogotá D.C.',
    'Distrito Capital de Bogotá': 'Bogotá D.C.',
    'Bogotá D.C.': 'Bogotá D.C.',
    'Bogotá D.C.': 'Bogotá D.C.',
    'San Andrés Providencia y Santa Catalina': 'San Andrés, Providencia y Santa Catalina',
    'San Andres': 'San Andrés, Providencia y Santa Catalina',
    'NORTE DE SANTANDER': 'Norte de Santander',
    'Norte De Santander': 'Norte de Santander',
    'Bolívar': 'Bolivar',
    'caldas': 'Caldas',
    'Caldas ': 'Caldas',
    'CAUCA': 'Cauca',
    'Valle del cauca': 'Valle del Cauca',
    'Cordoba': 'Córdoba',
    'cundinamarca': 'Cundinamarca',
    'LA GUAJIRA': 'La Guajira',
    'huila': 'Huila',
    'Huila ': 'Huila',
    'meta': 'Meta',
    'SANTANDER': 'Santander',
    'Santander ': 'Santander',
    'SUCRE': 'Sucre'
}

DF_Consulta['Departamento Entidad']=DF_Consulta['Departamento Entidad'].replace(reemplazos_departamento)
DF_Consulta['Departamento Entidad'].value_counts(dropna=False)

Departamento Entidad
Bogotá D.C.                                 829057
Antioquia                                   589886
Valle del Cauca                             376832
Cundinamarca                                277331
Nariño                                      222222
Santander                                   215514
Boyacá                                      164301
Magdalena                                   151001
Tolima                                      147400
Norte de Santander                          136889
Atlántico                                   133047
Huila                                       123321
Meta                                        118931
Caldas                                      117482
Bolivar                                     115658
Cauca                                        94710
Risaralda                                    89072
Quindío                                      81685
Cesar                                        72958
Sucre     

In [32]:
DF_Consulta['Departamento Entidad'].value_counts(dropna=False)

Departamento Entidad
Bogotá D.C.                                 829057
Antioquia                                   589886
Valle del Cauca                             376832
Cundinamarca                                277331
Nariño                                      222222
Santander                                   215514
Boyacá                                      164301
Magdalena                                   151001
Tolima                                      147400
Norte de Santander                          136889
Atlántico                                   133047
Huila                                       123321
Meta                                        118931
Caldas                                      117482
Bolivar                                     115658
Cauca                                        94710
Risaralda                                    89072
Quindío                                      81685
Cesar                                        72958
Sucre     

In [33]:
DF_Consulta['Ciudad Entidad']=DF_Consulta['Ciudad Entidad'].str.capitalize()

In [34]:
List_url=[]
for dic_url in DF_Consulta['URL']:
    List_url.append(dic_url['url'])
DF_Consulta['URL']=List_url

In [35]:
DF_Consulta.reset_index(drop=True, inplace=True)

In [36]:
DF_Consulta.to_parquet('../../../Sample_Data/Processed/SECOP_LIMPIO_2020_2023.parquet')

In [2]:
import pandas as pd
import os

In [3]:
DF_Consulta=pd.read_parquet('../../../Sample_Data/Processed/SECOP_LIMPIO_2020_2023.parquet')

In [4]:
from ydata_profiling import ProfileReport
### Reporte con pandas profiling
os.makedirs("../Reportes", exist_ok=True)
report=ProfileReport(DF_Consulta, title="Reporte de Exploración de Datos", 
                        minimal=True,dark_mode=True)

In [5]:
report.to_file("../Reportes/Reporte de Exploración de Datos.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/izainea/miniconda3/lib/python3.10/site-packages/ydata_profiling/model/pandas/describe_numeric_pandas.py:122: RuntimeWarning: overflow encountered in scalar subtract
  stats["range"] = stats["max"] - stats["min"]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Trabajo en datalake

En esta sección se establecen los parámetros de conexión al datalake y se cargan las librerías necesarias para la conexión y consulta de datos.

In [4]:
import pandas as pd
import pyarrow.parquet as pq
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# URL del punto final del servicio Blob y token SAS
url_blob = "https://adlindicadoresemae.blob.core.windows.net/"
token_sas = "?sv=2021-10-04&ss=btqf&srt=sco&st=2023-04-26T10%3A55%3A03Z&se=2023-07-27T10%3A55%3A00Z&sp=rwdxftlacup&sig=9%2FBvXPQRTnl4CUwGzrUpMFXbpu50nNRqix5pREicSiY%3D"

# Conexión a la cuenta de almacenamiento de Azure utilizando el token SAS
connect_str = f"BlobEndpoint={url_blob};SharedAccessSignature={token_sas}"
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [ ]:
# Referencia al contenedor donde se encuentran los archivos
container_name_ent = "datosemae/bronze/Entidades"
container_client_ent = blob_service_client.get_container_client(container_name_ent)

# Nombre del archivo a leer
blob_name_ent_small = "Entidades_SECOP_II.parquet"
blob_name_ent_full = "Entidades_SECOP_II_FULL.parquet"